## Домашнее задание № 2. Рейтинг-система

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from re import findall

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone # save best model at EM step
import scipy.stats as st
from scipy.sparse import lil_matrix, vstack # must have for EM
from itertools import product

#### Часть 1.

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
взять в тренировочный набор турниры с dateStart из 2019 года; 
в тестовый — турниры с dateStart из 2020 года.

In [2]:
tournaments_df = pd.read_pickle('tournaments.pkl')
train_ids = [v['id'] for k, v in tournaments_df.items() if v['dateStart'][:4] == '2019']
test_ids = [v['id'] for k, v in tournaments_df.items() if v['dateStart'][:4] == '2020']

results_df = pd.read_pickle('results.pkl')

Помимо общих условий, отбросим также результаты команд, для которых содержится неполная информация о матче (значения в mask, отличные от 0 и 1), а также те, длины маски которых отличаются от других (для нумерации соответствующих вопросов в рамках одного турнира):

In [3]:
results_train = {}
results_test = {}

for tournament_id, tournament_results in tqdm(results_df.items()):
    extracted_tournament_results = {}
    if tournament_id in train_ids or tournament_id in test_ids:
        for team_details in tournament_results:
            mask = team_details.get('mask')
            players = [player['player']['id'] for player in team_details['teamMembers']]

            # skip wrong records or having unknown token in mask
            if mask is None or not len(players) or findall('[^01]', mask):
                continue

            id = team_details['team']['id']
            team_name = team_details['team']['name']

            extracted_tournament_results[id] = {}
            extracted_tournament_results[id]['mask'] = mask
            extracted_tournament_results[id]['players'] = players
    
    # define max len of mask and remove teams scores that have shorter one
    mask_len_mapping = {
                            team_id: len(extracted_tournament_results[team_id]['mask'])
                            for team_id in extracted_tournament_results.keys()
                            }
    if len(extracted_tournament_results):                    
        max_mask_len = max(mask_len_mapping.values())
        for team_id in mask_len_mapping.keys():
            if len(extracted_tournament_results[team_id]['mask']) != max_mask_len:
                extracted_tournament_results.pop(team_id)
           
        if tournament_id in train_ids:
            results_train[tournament_id] = extracted_tournament_results
        elif tournament_id in test_ids:
            results_test[tournament_id] = extracted_tournament_results

del results_df

100%|██████████| 5528/5528 [00:00<00:00, 11394.08it/s]


#### Часть 2.

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Для начала преобразуем данные к табличному виду:

In [4]:
def process_baseline_dataset(results, init_question_id=0):
    df = []
    max_question_id = init_question_id
    for tournament_id, teams in tqdm(results.items()):
        tournament_question_count = None
        for team_id, team_details in teams.items():
            mask = np.array([int(ans) for ans in team_details['mask']])
            if tournament_question_count is None:
                tournament_question_count = len(mask)
            players = team_details['players']
            questions = np.tile(np.arange(max_question_id, max_question_id + tournament_question_count), len(players))
            answers = np.array(np.meshgrid(players, mask)).T.reshape(-1, 2)
            answers = np.hstack([
                                np.repeat(tournament_id, len(questions)).reshape(-1, 1),
                                np.repeat(team_id, len(questions)).reshape(-1, 1),
                                answers, 
                                questions.reshape(-1, 1)
                                ])
            df.append(answers)
        if tournament_question_count is not None:
            max_question_id += tournament_question_count
    df = np.vstack(df)
    return pd.DataFrame(
                        df, 
                        columns = ['tournament_id', 'team_id', 'player_id', 'answer', 'question_id']
                        )

Преобразуем трейн:

In [5]:
train_df = process_baseline_dataset(results_train)
train_df.head()

100%|██████████| 616/616 [00:05<00:00, 109.79it/s]


,tournament_id,team_id,player_id,answer,question_id
0,4772,45556,6212,1,0
1,4772,45556,6212,1,1
2,4772,45556,6212,1,2
3,4772,45556,6212,1,3
4,4772,45556,6212,1,4


Для обучения бэйзлайна нам потребуются лишь идентификаторы игроков, вопроса и результат.

In [6]:
ohe = OneHotEncoder(sparse=True, handle_unknown='ignore')
X_train = ohe.fit_transform(train_df[['player_id', 'question_id']])
y_train = train_df['answer']

Теперь можно обучить бэйзлайн модель:

In [7]:
logreg = LogisticRegression(random_state=42, n_jobs=-1)
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=42,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

Посмотрим распределение игроков по силе, отсортировав их по величине соответствующего коэффициента в линейной регрессии:

In [8]:
# extract player names
players_df = pd.read_pickle('players.pkl')
id_player_mapping = {}
for id in set(train_df.player_id):
    id_player_mapping[id] = players_df[id]['surname'] + ' ' + players_df[id]['name'] + ' ' + players_df[id]['patronymic']
del players_df

players_strength = pd.DataFrame({
    'player_id': ohe.categories_[0],
    'strength': logreg.coef_[0, :len(id_player_mapping)],
     })
players_strength['name'] = players_strength['player_id'].map(id_player_mapping)
players_strength.sort_values(by='strength', ascending=False, inplace=True)
players_strength.head(25)

,player_id,strength,name
3771,27403,3.702987,Руссо Максим Михайлович
585,4270,3.543858,Брутер Александра Владимировна
4141,30152,3.421965,Сорожкин Артём Сергеевич
5061,37047,3.411196,Юнгер Мария Алексеевна
3841,27822,3.292033,Савченков Михаил Владимирович
3957,28751,3.278940,Семушин Иван Николаевич
5242,38196,3.224670,Митрофанов Артём Александрович
53764,216863,3.208356,Гаврилов Глеб Юрьевич
6569,56647,3.199134,Горелова Наталья Евгеньевна
526,3843,3.185385,Бомешко Светлана Борисовна


Исходя из представленных рейтингов на официальном сайте, бэйзлайн прогноз кажется адекватным.

#### Часть 3.

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

По аналогии с трейном, преобразуем тест. Здесь стоит иметь ввиду, что вопросы заранее неизвестны, поэтому это поле стоит игнорировать, учитывая, что на качество ранжирования эта сложность в некотором смысле не влияет. Поэтому заменим все айди вопросов значением -1:

In [9]:
def process_baseline_dataset_test(results):
    NEW_QUESTION_INDEX = -1
    df = []
    for tournament_id, teams in tqdm(results.items()):
        tournament_question_count = None
        for team_id, team_details in teams.items():
            mask = np.array([int(ans) for ans in team_details['mask']])
            if tournament_question_count is None:
                tournament_question_count = len(mask)
            # take tournaments with same questions count
            if len(mask) == tournament_question_count:
                for player_id in team_details['players']:
                    df.append((tournament_id, team_id, player_id, NEW_QUESTION_INDEX, sum(mask)))
           
    df = np.vstack(df)
    return pd.DataFrame(
                        df, 
                        columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'score']
                        )

Преобразуем тест:

In [10]:
test_df = process_baseline_dataset_test(results_test)
test_df.head()

100%|██████████| 160/160 [00:00<00:00, 160.02it/s]


,tournament_id,team_id,player_id,question_id,score
0,5414,66120,18490,-1,33
1,5414,66120,116901,-1,33
2,5414,66120,8532,-1,33
3,5414,66120,42346,-1,33
4,5414,66120,123190,-1,33


Сделаем прогноз правильного ответа игрока в соответствии с бэйзлайн моделью:

In [11]:
X_test = ohe.transform(test_df[['player_id', 'question_id']])
success_pred = logreg.predict_proba(X_test)[:, 1]

Для построения рейтинга команд были рассмотрены следующие подходы:

1) ранжирование команд исходя из вероятности того, что хотя бы один игрок команды правильно ответит на вопрос;

2) учитывая, что в командах сравнительно мало (как правило менее 6) игроков, сила команда оценивалась как вероятность того, что какой-то (фиксированный) процент игроков команды правильно ответит на вопрос;

Далее реализован расчет указанных метрик и целевых статистик.

In [12]:
def someone_answer(x):
    return 1 - np.product(1 - x)

def team_vote(x, bound=0.2):
    dist = np.array(x)
    res = 0
    cnt = len(dist)
    bound = np.ceil(bound * cnt)
    for ans in product([0, 1], repeat=cnt):
        if not np.sum(ans) < bound:
            ans = np.array(ans)
            p_ans = np.product(dist ** ans) * np.product((1 - dist) ** (1 - ans))
            res += p_ans
    return res

def get_score(df, pred, agg_func=someone_answer):
    df['pred'] = pred
    df['pred'] = df.groupby(['tournament_id', 'team_id'])['pred'].transform(lambda x: agg_func(x))
    rate_df = df[['tournament_id', 'team_id', 'score', 'pred']].drop_duplicates().reset_index(drop=True)

    rate_df = rate_df.sort_values(by=['tournament_id', 'pred'], ascending=False)
    rate_df['pred_rank'] = rate_df.groupby('tournament_id')['pred'].transform(lambda x: np.arange(1, len(x) + 1))

    rate_df = rate_df.sort_values(by=['tournament_id', 'score'], ascending=False)
    rate_df['true_rank'] = rate_df.groupby('tournament_id')['score'].transform(lambda x: np.arange(1, len(x) + 1))

    rate_df.drop(['pred', 'score'], axis=1, inplace=True)
    rho = rate_df.groupby('tournament_id').apply(lambda x: st.spearmanr(x['true_rank'], x['pred_rank']).correlation).mean()
    tau = rate_df.groupby('tournament_id').apply(lambda x: st.kendalltau(x['true_rank'], x['pred_rank']).correlation).mean()
    return rho, tau

Посчитаем качество ранжирования для каждой из стратегий (первое число - корреляция Спирмена, второе - Кендалла):

In [13]:
%%time
get_score(test_df, success_pred)

Wall time: 7.08 s


(0.7705446620836607, 0.6204650001036078)

In [14]:
%%time
get_score(test_df, success_pred, agg_func=team_vote)

Wall time: 29.8 s


(0.7097367202909768, 0.5526203337533793)

Корреляции в норме для каждого из вариантов, однако подход основанный на вероятности ответа хотя бы одного игрока показывает лучшие результаты (рассматривались различные значения bound) по корреляциям, а также значительно легче обобщается для бОльшего числа игроков.

#### Часть 4.

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
разработайте EM-схему для обучения этой модели, реализуйте её в коде;
обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

Будем считать, что команда дает правильный ответ, если хотя бы один из ее игроков знает правильный ответ.
Теперь введем скрытые переменные $z_{ij}$, которые равны 1, если игрок $i$ правильно ответил на вопрос $j$, и нулю в противном случае.

Тогда, если команда ответила на вопрос верно:
$$P(z_{ij}| y=1)=\frac{P(y=1|z_{ij}) \cdot P(z_{ij})}{P(y=1)},$$
где $y$ - ответ на вопрос, а $P(y=1)=1 - \prod\limits_{i = 1}^{n_{players}} p(z_{ij})$.

Для обучения предложенной модели ЕМ-алгоритмом будет как и раньше использовать логистическую регрессию, добавив веса для каждого "наблюдения". Тогда:

- на Е шаге будем обновлять веса регрессии, используя выражение для $P(z_{ij}| y=1)$;
- на М шаге будет переобучать логистическую регрессию используя полученные ранее веса.

In [15]:
X = vstack([X_train, X_train], format='lil', dtype=np.int8)
Y = (X_train.shape[0] * [1] + X_train.shape[0] * [0])

In [16]:
class EMclassifier:
    def __init__(self):
        self.model = LogisticRegression(random_state=42, n_jobs=-1)
        print("Model initializing...\n")
        self.model.fit(X_train, y_train)
        self.pred = None
        self.best_model = None
        self.best_rho = -1
        self.best_tau = -1

    def __E_step(self):
        self.pred = self.model.predict_proba(X_train)[:, 1]
        self.pred = self.__weight_update(results_train, self.pred)
    
    def __M_step(self):
        lr_weights = np.hstack([self.pred, 1 - self.pred])
        self.model.fit(X, Y, sample_weight = lr_weights)

    def fit(self, n_steps=5):
        for n_step in range(n_steps):
            print("\nStep ", n_step + 1," ...\n")
            self.__E_step()
            self.__M_step()
            next_score = self.model.predict_proba(X_test)[:, 1]
            rho, tau = get_score(test_df, next_score)
            print("\nSpearman correlation: ", rho)
            print("Kendall correlation: ", tau)
            if rho > self.best_rho:
                self.best_model = clone(self.model)
                self.best_model.coef_ = self.model.coef_
                self.best_rho = rho
                self.best_tau = tau

    @staticmethod
    def __weight_update(results, pred):
        pos = 0
        next_pred = []
        for _, teams in tqdm(results.items()):
            for team_id, team_details in teams.items():
                mask = np.array([int(ans) for ans in team_details['mask']])
                question_count = len(mask)
                players = team_details['players']
                players_count = len(players)
                batch_size = question_count * len(players)
                batch = pred[pos : pos + batch_size].reshape(players_count, question_count)
                updated_batch = np.clip(batch / (1 - np.product(1 - batch, axis=0)) * mask, 0, 1)
                next_pred += updated_batch.reshape(1, batch_size).tolist()[0]
                pos += batch_size
        return np.array(next_pred)

In [17]:
em = EMclassifier()
em.fit()

Model initializing...


Step  1  ...

100%|██████████| 616/616 [00:04<00:00, 140.59it/s]

Spearman correlation:  0.7712751309309436
Kendall correlation:  0.6224694786250812

Step  2  ...

100%|██████████| 616/616 [00:04<00:00, 142.60it/s]

Spearman correlation:  0.7729747264753636
Kendall correlation:  0.6237751933022099

Step  3  ...

100%|██████████| 616/616 [00:04<00:00, 144.25it/s]

Spearman correlation:  0.7659632663373849
Kendall correlation:  0.6171607554984493

Step  4  ...

100%|██████████| 616/616 [00:04<00:00, 143.64it/s]

Spearman correlation:  0.7770891992593054
Kendall correlation:  0.628093722834591

Step  5  ...

100%|██████████| 616/616 [00:04<00:00, 133.58it/s]

Spearman correlation:  0.779524943450047
Kendall correlation:  0.6303400709880445


Как видно из результатов расчета, использование ЕМ алгоритма примерно на 1% увеличило качество ранжирования.

#### Часть 5.

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)?

Оценим "сложность" турнира медианой сложности вопроса, разыгранного в турнире (в данном случае, коэффициента логистической регрессии, соответствующего question_id). Извлечем данные и выведем топ-20 "сложных" и "простых" турниров:

In [18]:
train_questions_count = max(train_df.question_id) + 1
question_to_weight_mapping = pd.DataFrame({
    'question_id': np.arange(train_questions_count),
    'weight': em.best_model.coef_[0, -train_questions_count:]
     })

tournament_rate_df = pd.merge(train_df[['tournament_id', 'question_id']], question_to_weight_mapping, how='left', on='question_id')
tournament_rate_df = tournament_rate_df.groupby('tournament_id')['weight'].median().reset_index()

names = []
for id in tournament_rate_df.tournament_id:
    names.append(tournaments_df[id]['name'])
    
tournament_rate_df['tournament_name'] = names
tournament_rate_df.sort_values(by='weight', inplace=True)

In [19]:
tournament_rate_df.head(20)

,tournament_id,weight,tournament_name
35,5159,-2.053856,Первенство правого полушария
488,5928,-2.032680,Угрюмый Ёрш
604,6149,-1.942270,Чемпионат Санкт-Петербурга. Первая лига
539,5996,-1.613044,Тихий Донец: омут первый
254,5587,-1.565750,Записки охотника
58,5303,-1.564225,Мемориал Дмитрия Коноваленко
12,5025,-1.544080,Кубок городов
36,5161,-1.531693,Антибинго
580,6101,-1.531433,Воображаемый музей
157,5465,-1.529842,Чемпионат России


In [20]:
tournament_rate_df.tail(20)

,tournament_id,weight,tournament_name
174,5486,1.193579,Маленькае люстэрка
8,5011,1.225942,(а)Синхрон-lite. Лига старта. Эпизод IV
351,5704,1.243747,(а)Синхрон-lite. Лига старта. Эпизод X
519,5972,1.244274,Кубок малых городов
345,5698,1.248929,(а)Синхрон-lite. Лига старта. Эпизод VII
443,5857,1.255784,Летний салат
442,5855,1.287211,Лига вузов. IV тур
196,5511,1.289171,KFC
423,5828,1.290265,Гарри Поттер и 3 по 12
33,5130,1.314238,Лига Сибири. VI тур.


В целом, посчитанный рейтинг соответствует интуиции. В топе есть турниры уровня Чемпионата Санкт-Петербурга, Кубка Москвы и Чемпионата Мира, а в конце - группа турниров формата "Лига Старта".